In [1]:
import xarray as xr

In [2]:
ds = xr.tutorial.open_dataset('air_temperature')

In [4]:
ds.nbytes/1e9

0.015499672

In [7]:
import os

# Define the environment variable
os.environ['MALLOC_TRIM_THRESHOLD_'] = '0'

In [9]:
client.close()

In [ ]:
from dask.distributed import Client
#client = Client(threads_per_worker=1)
client = Client(client_kwargs={'env': {'MALLOC_TRIM_THRESHOLD_': '0'}})
client

2024-04-04 13:50:56,842 - distributed.nanny - ERROR - Failed to initialize worker
Traceback (most recent call last):
  File "/Users/moo270/mambaforge/envs/pangeo_rechunker/lib/python3.12/site-packages/distributed/nanny.py", line 954, in run
    worker = worker_factory()
             ^^^^^^^^^^^^^^^^
  File "/Users/moo270/mambaforge/envs/pangeo_rechunker/lib/python3.12/site-packages/distributed/worker.py", line 716, in __init__
    ServerNode.__init__(
TypeError: Server.__init__() got an unexpected keyword argument 'client_kwargs'
2024-04-04 13:50:56,842 - distributed.nanny - ERROR - Failed to initialize worker
Traceback (most recent call last):
  File "/Users/moo270/mambaforge/envs/pangeo_rechunker/lib/python3.12/site-packages/distributed/nanny.py", line 954, in run
    worker = worker_factory()
             ^^^^^^^^^^^^^^^^
  File "/Users/moo270/mambaforge/envs/pangeo_rechunker/lib/python3.12/site-packages/distributed/worker.py", line 716, in __init__
    ServerNode.__init__(
TypeErro

In [6]:
ds = xr.tutorial.open_dataset('air_temperature')